# Lab: Vertex AI–Assisted BigQuery Analytics — Example Prompts
**Goal:** Practice moving from simple SQL to complex analytics in BigQuery using *only* carefully engineered prompts with Vertex AI (Gemini).  
**Important:** This notebook contains **prompts only** (no starter code). Paste the prompts into **Vertex AI Studio**, **Vertex AI in Colab Enterprise**, or your chosen chat interface, and then run the generated SQL directly in **BigQuery**. If you decide to automate later, you can ask Vertex AI to convert the winning SQL into a Colab pipeline.

## How to use this prompts-only notebook
1. Open **Vertex AI Studio** (or Gemini in Colab Enterprise chat panel).  
2. Copy a prompt from this notebook and paste it into the model. Do **not** paste any code from here; let the model generate it.  
3. Run the generated SQL in **BigQuery** (Console → BigQuery Studio).  
4. Iterate: refine the prompt when results aren’t what you expect.  
5. Document: capture your final SQL, plus a one-sentence takeaway, in your notes/README.

## Dataset assumptions
Use one of these sources (adjust table paths accordingly):
- **Global Superstore (Kaggle)** loaded into BigQuery (e.g., `[YOUR_PROJECT].superstore_data.sales`)  
- **TheLook eCommerce** public dataset: `bigquery-public-data.thelook_ecommerce`  
If you are using *Global Superstore*, make sure column names match your schema (e.g., `Order_Date`, `Region`, `Category`, `Sub_Category`, `Sales`, `Profit`, `Discount`, `State`, `Customer_ID`, `Ship_Mode`).

---
## Prompting guardrails (quick checklist)
- **Be explicit**: table path, column names, filters, output columns, sort order, and limits.  
- **Ask for runnable SQL**: “Return a BigQuery SQL block only.”  
- **Control cost**: ask for `LIMIT` during exploration and remove it for the final run.  
- **Validate**: request a brief explanation of why each clause is present and how you can sanity-check results.
---

## Install Dependencies

In [3]:
# Install the Google Cloud BigQuery client library
!pip install google-cloud-bigquery==3.17.0 pandas==2.1.4

# Authenticate your Colab environment
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.2/230.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 92.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: google-cloud-bigquery
    Found existing installation: google-cloud-bigquery 3.38.0
    Uninstalling google-cloud-bigquery-3.38.0:
      Successfully uninstalled google-cloud-bigquery-3.38.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency con

Authenticated


## Copy Schema to a dataframe

In [1]:
from google.cloud import bigquery
import pandas as pd

# Replace with your Google Cloud Project ID
project_id = 'upbeat-aspect-471118-v8' # This is derived from your provided table name
dataset_id = 'lab01_dataset'
table_id = 'Superstore'

# Construct a BigQuery client object.
client = bigquery.Client(project=project_id)

# Get the table object
table_ref = client.dataset(dataset_id).table(table_id)
table = client.get_table(table_ref)

# Extract schema information
schema_list = []
for field in table.schema:
    schema_list.append({
        'name': field.name,
        'field_type': field.field_type,
        'mode': field.mode,
        'description': field.description
    })

# Convert to Pandas DataFrame
schema_df = pd.DataFrame(schema_list)

# Display the schema DataFrame (optional, for verification)
print("Schema DataFrame created:")
# To see the output, run the code.


Schema DataFrame created:


## CLean Column Names

In [2]:
# --- 1. Clean the Column Names ---
# Create a 'clean_name' column with standard naming conventions:
# lowercase, with spaces and hyphens replaced by underscores.
schema_df['clean_name'] = schema_df['name'].str.lower().str.replace(' ', '_').str.replace('-', '_')


# --- 2. Generate the Aliases for the SELECT Clause ---
column_expressions = []
for index, row in schema_df.iterrows():
    original_name = row['name']
    clean_name = row['clean_name']

    # If the original name contains a space or special character, it needs to be
    # enclosed in backticks (`) in the SQL statement.
    if ' ' in original_name or '-' in original_name:
        expression = f'`{original_name}` AS {clean_name}'
    else:
        # If the name is already clean, we still alias it for consistency.
        expression = f'{original_name} AS {clean_name}'
    column_expressions.append(expression)

# Join all the individual column expressions into a single, formatted string.
select_clause = ",\n  ".join(column_expressions)


# --- 3. Construct the Final CREATE VIEW Statement ---
new_view_id = 'superstore_clean' # You can change this if you like

create_view_sql = f"""
CREATE OR REPLACE VIEW `{project_id}.{dataset_id}.{new_view_id}` AS
SELECT
  {select_clause}
FROM
  `{project_id}.{dataset_id}.{table_id}`;
"""

# --- 4. Print the Final SQL ---
print("--- Copy the SQL below and run it in your BigQuery Console ---")
print(create_view_sql)

--- Copy the SQL below and run it in your BigQuery Console ---

CREATE OR REPLACE VIEW `upbeat-aspect-471118-v8.lab01_dataset.superstore_clean` AS
SELECT
  `Row ID` AS row_id,
  `Order ID` AS order_id,
  `Order Date` AS order_date,
  `Ship Date` AS ship_date,
  `Ship Mode` AS ship_mode,
  `Customer ID` AS customer_id,
  `Customer Name` AS customer_name,
  Segment AS segment,
  Country AS country,
  City AS city,
  State AS state,
  `Postal Code` AS postal_code,
  Region AS region,
  `Product ID` AS product_id,
  Category AS category,
  `Sub-Category` AS sub_category,
  `Product Name` AS product_name,
  Sales AS sales,
  Quantity AS quantity,
  Discount AS discount,
  Profit AS profit
FROM
  `upbeat-aspect-471118-v8.lab01_dataset.Superstore`;



## Generate View with standard column naming convention

In [3]:
# Execute the CREATE VIEW SQL query
try:
    query_job = client.query(create_view_sql)  # API request
    query_job.result()  # Waits for the query to finish
    print(f"View '{new_view_id}' created/replaced successfully in dataset '{dataset_id}'.")
except Exception as e:
    print(f"An error occurred while creating the view: {e}")

# Now, let's print 10 rows from the newly created view to verify
print(f"\n--- First 10 rows from the new view '{new_view_id}' ---")
try:
    # Construct a reference to the new view
    view_table_ref = client.dataset(dataset_id).table(new_view_id)

    # Fetch the first 10 rows
    rows = client.list_rows(view_table_ref, max_results=10)

    # Print header
    print(" | ".join([field.name for field in rows.schema]))
    print("-" * 80) # Separator

    # Print rows
    for row in rows:
        print(" | ".join([str(item) for item in row.values()]))

except Exception as e:
    print(f"An error occurred while fetching rows from the view: {e}")



View 'superstore_clean' created/replaced successfully in dataset 'lab01_dataset'.

--- First 10 rows from the new view 'superstore_clean' ---
row_id | order_id | order_date | ship_date | ship_mode | customer_id | customer_name | segment | country | city | state | postal_code | region | product_id | category | sub_category | product_name | sales | quantity | discount | profit
--------------------------------------------------------------------------------
An error occurred while fetching rows from the view: 400 GET https://bigquery.googleapis.com/bigquery/v2/projects/upbeat-aspect-471118-v8/datasets/lab01_dataset/tables/superstore_clean/data?maxResults=10&formatOptions.useInt64Timestamp=True&prettyPrint=false: Cannot list a table of type VIEW.


In [4]:
# This assumes your 'client' object from the previous cell is still active
# and correctly authenticated.

print("✅ Step 1: Defining the query string...")

query_string = """
SELECT
  order_id,
  customer_name,
  product_name,
  sales,
  profit
FROM
  `upbeat-aspect-471118-v8.lab01_dataset.superstore_clean`
LIMIT 10;
"""

print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

# Use a try-except block to catch potential errors
try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df = query_job.to_dataframe()

    print(f"✅ Step 4: Query finished. Found {len(results_df)} rows.")

    if results_df.empty:
        print("\n⚠️ The query ran successfully but returned an empty result. Please double-check that your 'superstore_clean' view exists and the original table has data.")
    else:
        print("\n--- Displaying Results ---")
        display(results_df)

except Exception as e:
    print(f"\n❌ An error occurred: {e}")

✅ Step 1: Defining the query string...
✅ Step 2: Sending the query to BigQuery. This may take a moment...
✅ Step 3: Waiting for query to complete and fetching results...
✅ Step 4: Query finished. Found 10 rows.

--- Displaying Results ---


,order_id,customer_name,product_name,sales,profit
0,CA-2015-154900,Sung Shariari,Avery 518,3.15,1.5120
1,CA-2015-154900,Sung Shariari,Adams Telephone Message Book W/Dividers/Space ...,22.72,10.2240
2,US-2016-152415,Patrick O'Donnell,"C-Line Magnetic Cubicle Keepers, Clear Polypro...",14.82,6.2244
3,US-2016-152415,Patrick O'Donnell,"Howard Miller 14-1/2"" Diameter Chrome Round Wa...",191.82,61.3824
4,CA-2016-153269,Pamela Stobb,"Personal Folder Holder, Ebony",11.21,3.3630
5,CA-2016-153269,Pamela Stobb,"Situations Contoured Folding Chairs, 4/Set",354.90,88.7250
6,CA-2016-153269,Pamela Stobb,Xerox 193,17.94,8.7906
7,CA-2016-153269,Pamela Stobb,GBC Binding covers,51.80,23.3100
8,CA-2015-158792,Brian Dahlen,Staples,22.20,10.4340
9,CA-2016-141082,Fred McMath,Avery 517,3.69,1.7343


## Part A — SQL Warm‑Up (SELECT, WHERE, ORDER BY, LIMIT, DISTINCT)
**Aim:** Build confidence with precise, unambiguous prompts that yield clean, runnable SQL.

### A1. Unique values (DISTINCT)
**Prompt (paste in Vertex AI):**
```
Act as a senior BigQuery analyst. Produce a **single runnable BigQuery SQL** (no commentary) for:
- Task: List all unique `Sub_Category` values sold in the 'West' region.
- Table: `mgmt-467-47888.lab1_foundation.superstore`
- Filter: `Region = 'West'`
- Output: a single column named `Sub_Category`
- Sort: alphabetically A→Z
- Add: `LIMIT 100` to control cost during exploration.
```
**Reflection:** Did the result match your expectations? If not, what ambiguity in your prompt might have caused the mismatch?

My result matched the query below and met my most of my expectations for what the code should look like. I should have specified how I wanted the col of Sub-Category to be ordered. Next time I should include information about how the order by should be ASC or DESC


In [ ]:
query_string = """
SELECT
    DISTINCT sub_category AS Sub_Category
FROM
    `upbeat-aspect-471118-v8.lab01_dataset.superstore_clean`
WHERE
    Region = 'West'
ORDER BY
    Sub_Category ASC
LIMIT 100
"""
query_job = client.query(query_string) # Re-run the query to get updated results
results_df = query_job.to_dataframe()
display(results_df)

,Sub_Category
0,Accessories
1,Appliances
2,Art
3,Binders
4,Bookcases
5,Chairs
6,Copiers
7,Envelopes
8,Fasteners
9,Furnishings


### A2. Top‑N by metric (ORDER BY … DESC)
**Prompt:**
```
BigQuery SQL only.
Task: Return the top 10 customers by total profit.
Table: `mgmt-467-47888.lab_foundation.superstore`
Columns used: `Customer_ID`, `Profit`
Output columns: `Customer_ID`, `total_profit`
Logic: SUM Profit per customer, order by `total_profit` DESC
Add `LIMIT 10`.
```
**Tip:** If your schema uses different identifiers (e.g., `Customer Name`), restate column names explicitly.

In [ ]:
query_string = """
SELECT
    customer_id,
    SUM(profit) AS total_profit
FROM
    `upbeat-aspect-471118-v8.lab01_dataset.superstore_clean`
GROUP BY
    customer_id
ORDER BY
    total_profit DESC
LIMIT 10
"""
query_job = client.query(query_string) # Re-run the query to get updated results
results_df = query_job.to_dataframe()
display(results_df)

,customer_id,total_profit
0,TC-20980,8981.3239
1,RB-19360,6976.0959
2,SC-20095,5757.4119
3,HL-15040,5622.4292
4,AB-10105,5444.8055
5,TA-21385,4703.7883
6,CM-12385,3899.8904
7,KD-16495,3038.6254
8,AR-10540,2884.6208
9,DR-12940,2869.0760


### A3. Basic filtering (WHERE) + sanity checks
**Prompt:**
```
BigQuery SQL only.
Task: Count orders shipped with each `Ship_Mode`, but only for orders in the 'Technology' category.
Table: `[YOUR_PROJECT].superstore_data.sales`
Output: `Ship_Mode`, `order_count`
Logic: COUNT(*) grouped by `Ship_Mode`
Sort by `order_count` DESC
```
**Validation ask:** “Also list two quick sanity checks to verify the numbers.”

In [ ]:
sql = f"""
SELECT
    Ship_Mode,
    COUNT(*) AS order_count
FROM
    `upbeat-aspect-471118-v8.lab01_dataset.superstore_clean`
WHERE
    Category = 'Technology'
GROUP BY
    Ship_Mode
ORDER BY
    order_count DESC
"""

# Execute the query
query_job = client.query(sql)
results_df = query_job.to_dataframe()
display(results_df)

,Ship_Mode,order_count
0,Standard Class,1082
1,Second Class,366
2,First Class,301
3,Same Day,98


## Part B — Grouped Analytics (GROUP BY, HAVING)
**Aim:** Turn raw facts into grouped metrics and filtered aggregations.

### B1. KPI aggregation with WHERE + GROUP BY
**Prompt:**
```
BigQuery SQL only.
Task: Compute monthly revenue for the last 12 full months.
Table: `[YOUR_PROJECT].superstore_data.sales`
Assume: `Order_Date` is a DATE or TIMESTAMP column named exactly `Order_Date`.
Output: `year_month` (YYYY-MM format), `monthly_revenue`
Logic: Truncate date to month, SUM `Sales`, filter to last 12 full months.
Sort by `year_month` ascending.
Include a `LIMIT` safeguard for exploration.
```

In [ ]:
sql = f"""
SELECT
    FORMAT_DATE('%Y-%m', order_date) AS year_month,
    SUM(sales) AS monthly_revenue
FROM
    `upbeat-aspect-471118-v8.lab01_dataset.superstore_clean`
WHERE
    EXTRACT(YEAR FROM order_date) = 2017
GROUP BY
    year_month
ORDER BY
    year_month ASC
"""

# Execute the query
query_job = client.query(sql)

# Convert results to a Pandas DataFrame for easy display
results_df = query_job.to_dataframe()

# Display the DataFrame in Colab
display(results_df)

,year_month,monthly_revenue
0,2017-01,43971.3740
1,2017-02,20301.1334
2,2017-03,58872.3528
3,2017-04,36521.5361
4,2017-05,44261.1102
5,2017-06,52981.7257
6,2017-07,45264.4160
7,2017-08,63120.8880
8,2017-09,87866.6520
9,2017-10,77776.9232


### B2. Post‑aggregation filter (HAVING)
**Prompt:**
```
BigQuery SQL only.
Task: Find sub-categories whose total profit over the entire dataset is negative.
Table: `[YOUR_PROJECT].superstore_data.sales`
Output: `Sub_Category`, `total_profit`
Logic: SUM `Profit` GROUP BY `Sub_Category`, HAVING SUM(Profit) < 0
Sort by `total_profit` ASC (most negative first).
```
**Why HAVING?** Ask the model to include a 1-sentence explanation of why HAVING is used instead of WHERE here.

In [ ]:
sql = f"""
SELECT
    sub_category,
    SUM(profit) AS total_profit
FROM
    `upbeat-aspect-471118-v8.lab01_dataset.superstore_clean`
GROUP BY
    sub_category
HAVING
    SUM(profit) < 0
ORDER BY
    total_profit ASC
"""

# Execute the query
query_job = client.query(sql)

# Convert results to a Pandas DataFrame for easy display
results_df = query_job.to_dataframe()

# Display the DataFrame in Colab
display(results_df)

,sub_category,total_profit
0,Tables,-17725.4811
1,Bookcases,-3472.5560
2,Supplies,-1189.0995


## Part C — Joins (dimension enrichment)
**Aim:** Use joins to enhance facts with attributes.

### C1. Join facts to a small dimension
*(If you have a customer or product dimension in your schema, use it. Otherwise, request a synthetic example.)*  
**Prompt:**
```
BigQuery SQL only.
Task: Join the sales table to a product dimension to report `Product_ID`, `Product_Name`, and total sales.
Tables: `[YOUR_PROJECT].superstore_data.sales` as s, `[YOUR_PROJECT].superstore_data.products` as p
Join key: `s.Product_ID = p.Product_ID`
Output: `Product_ID`, `Product_Name`, `total_sales`
Sort by `total_sales` DESC
```
**If you lack a dimension table:** Ask the model how to simulate one temporarily via a CTE.

In [ ]:
sql = f"""
SELECT
    s.product_id,
    p.product_name,
    SUM(s.sales) AS total_sales
FROM
    `{project_id}.{dataset_id}.superstore_clean` AS s
JOIN
    `{project_id}.{dataset_id}.superstore_clean` AS p ON s.product_id = p.product_id
GROUP BY
    s.product_id,
    p.product_name
ORDER BY
    total_sales DESC
"""

# Execute the query
query_job = client.query(sql)

# Convert results to a Pandas DataFrame for easy display
results_df = query_job.to_dataframe()

# Display the DataFrame in Colab
display(results_df)

,product_id,product_name,total_sales
0,TEC-CO-10004722,Canon imageCLASS 2200 Advanced Copier,307999.120
1,OFF-BI-10003527,Fellowes PB500 Electric Punch Plastic Comb Bin...,274533.840
2,OFF-BI-10001359,GBC DocuBind TL300 Electric Binding System,218058.269
3,FUR-CH-10002024,HON 5400 Series Task Chairs for Big and Tall,174964.608
4,OFF-BI-10000545,GBC Ibimaster 500 Manual ProClick Binding System,171220.500
...,...,...,...
1889,OFF-AR-10003986,Avery Hi-Liter Pen Style Six-Color Fluorescent...,7.700
1890,OFF-EN-10001535,Grip Seal Envelopes,7.072
1891,OFF-PA-10000048,Xerox 20,6.480
1892,OFF-LA-10003388,Avery 5,5.760


In [ ]:
sql = f"""
WITH SimulatedProducts AS (
    -- This CTE acts as our 'products' dimension table
    SELECT
        CAST(product_id AS STRING) AS product_id, -- Ensure data type matches for join
        product_name
    FROM
        UNNEST([
            STRUCT('TEC-CO-10004722' AS product_id, 'Canon imageCLASS 2200 Advanced Copier' AS product_name),
            STRUCT('OFF-BI-10003527' AS product_id, 'Fellowes PB500 Electric Punch Plastic Comb Binding Machine' AS product_name),
            STRUCT('OFF-BI-10001359' AS product_id, 'GBC DocuBind TL300 Electric Binding System' AS product_name),
            STRUCT('FUR-CH-10002024' AS product_id, 'HON 5400 Series Task Chairs for Big and Tall' AS product_name),
            STRUCT('OFF-BI-10000545' AS product_id, 'GBC Ibimaster 500 Manual ProClick Binding System' AS product_name)
        ]) AS ProductsData
)
SELECT
    s.product_id,
    sp.product_name,
    SUM(s.sales) AS total_sales
FROM
    `upbeat-aspect-471118-v8.lab01_dataset.superstore_clean` AS s
JOIN
    SimulatedProducts AS sp ON s.product_id = sp.product_id
GROUP BY
    s.product_id,
    sp.product_name
ORDER BY
    total_sales DESC
"""

# Execute the query
query_job = client.query(sql)

# Convert results to a Pandas DataFrame for easy display
results_df = query_job.to_dataframe()

# Display the DataFrame in Colab
display(results_df)

,product_id,product_name,total_sales
0,TEC-CO-10004722,Canon imageCLASS 2200 Advanced Copier,61599.824
1,OFF-BI-10003527,Fellowes PB500 Electric Punch Plastic Comb Bin...,27453.384
2,FUR-CH-10002024,HON 5400 Series Task Chairs for Big and Tall,21870.576
3,OFF-BI-10001359,GBC DocuBind TL300 Electric Binding System,19823.479
4,OFF-BI-10000545,GBC Ibimaster 500 Manual ProClick Binding System,19024.500


## Part D — Common Table Expressions (CTEs)
**Aim:** Make complex logic readable and testable in steps.

### D1. Multi‑step ranking with CTEs
**Prompt:**
```
BigQuery SQL only.
Goal: Within each `Region`, rank states by total sales and return top 3 per region.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE 1 (`state_sales`): SUM(Sales) by `Region`, `State`
CTE 2 (`ranked_state_sales`): Add `RANK() OVER (PARTITION BY Region ORDER BY total_sales DESC)` as `sales_rank`
Final SELECT: rows where `sales_rank <= 3`
Output columns: `Region`, `State`, `total_sales`, `sales_rank`
Sort: by `Region`, then `sales_rank`
```
**Ask for**: a one-paragraph explanation of each step, then **provide only the final runnable SQL**.

In [ ]:
sql = f"""
WITH state_sales AS (
    -- CTE 1: Calculate total sales for each state within each region
    SELECT
        Region,
        State,
        SUM(Sales) AS total_sales
    FROM
        `upbeat-aspect-471118-v8.lab01_dataset.superstore_clean`
    GROUP BY
        Region,
        State
),
ranked_state_sales AS (
    -- CTE 2: Rank states within each region by their total sales
    SELECT
        Region,
        State,
        total_sales,
        RANK() OVER (PARTITION BY Region ORDER BY total_sales DESC) AS sales_rank
    FROM
        state_sales
)
-- Final SELECT: Filter to get the top 3 states per region
SELECT
    Region,
    State,
    total_sales,
    sales_rank
FROM
    ranked_state_sales
WHERE
    sales_rank <= 3
ORDER BY
    Region ASC,
    sales_rank ASC
"""

# Execute the query
query_job = client.query(sql)

# Convert results to a Pandas DataFrame for easy display
results_df = query_job.to_dataframe()

# Display the DataFrame in Colab
display(results_df)

,Region,State,total_sales,sales_rank
0,Central,Texas,170188.0458,1
1,Central,Illinois,80166.1010,2
2,Central,Michigan,76269.6140,3
3,East,New York,310876.2710,1
4,East,Pennsylvania,116511.9140,2
5,East,Ohio,78258.1360,3
6,South,Florida,89473.7080,1
7,South,Virginia,70636.7200,2
8,South,North Carolina,55603.1640,3
9,West,California,457687.6315,1


### D2. Time‑boxed “most improved” analysis
**Prompt:**
```
BigQuery SQL only.
Goal: Identify the top 5 sub-categories with the largest YoY revenue increase from 2023 to 2024.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE `yr_sales`: SUM(Sales) by `Sub_Category` and `year` extracted from `Order_Date`
Final: pivot or self-join to compute delta (2024 minus 2023) as `yoy_delta`
Output: `Sub_Category`, `sales_2023`, `sales_2024`, `yoy_delta`
Order by `yoy_delta` DESC
Limit 5
```
**Validation:** Ask the model for two quick failure modes (e.g., missing years) and how to handle them.

In [ ]:
sql = f"""
WITH yr_sales AS (
    -- CTE 1: Calculate total sales for each sub-category by year
    SELECT
        sub_category,
        EXTRACT(YEAR FROM order_date) AS order_year,
        SUM(sales) AS total_sales
    FROM
        `upbeat-aspect-471118-v8.lab01_dataset.superstore_clean`
    WHERE
        EXTRACT(YEAR FROM order_date) IN (2016, 2017) -- Filter for relevant years in your data
    GROUP BY
        sub_category,
        order_year
)
-- Final SELECT: Pivot or self-join to compute YoY delta and get top 5
SELECT
    s2016.sub_category,
    s2016.total_sales AS sales_2016,
    COALESCE(s2017.total_sales, 0) AS sales_2017, -- Use COALESCE to handle cases where a sub-category might not exist in 2017
    COALESCE(s2017.total_sales, 0) - s2016.total_sales AS yoy_delta
FROM
    yr_sales AS s2016
LEFT JOIN
    yr_sales AS s2017 ON s2016.sub_category = s2017.sub_category AND s2017.order_year = 2017 AND s2016.order_year = 2016
WHERE
    s2016.order_year = 2016 -- Ensure we are starting from 2016 data for the comparison
ORDER BY
    yoy_delta DESC
LIMIT 5
"""

# Execute the query
query_job = client.query(sql)

# Convert results to a Pandas DataFrame for easy display
results_df = query_job.to_dataframe()

# Display the DataFrame in Colab
display(results_df)

,sub_category,sales_2016,sales_2017,yoy_delta
0,Phones,78962.030,105340.516,26378.486
1,Binders,49683.325,72788.045,23104.720
2,Accessories,41895.854,59946.232,18050.378
3,Appliances,26050.315,42926.932,16876.617
4,Copiers,49599.410,62899.388,13299.978


## Part E — Window Functions (ROW_NUMBER, RANK, DENSE_RANK, LAG/LEAD, moving averages)
**Aim:** Compare rows across partitions and time; compute trends and ranks without collapsing rows.

### E1. Top product per region (ROW_NUMBER)
**Prompt:**
```
BigQuery SQL only.
Task: For each `Region`, return only the single highest-revenue `Sub_Category`.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE `subcat_sales`: SUM(Sales) by `Region`, `Sub_Category`
Add `ROW_NUMBER() OVER (PARTITION BY Region ORDER BY total_sales DESC)` as rn
Final: filter `rn = 1`
Output: `Region`, `Sub_Category`, `total_sales`
Sort by `Region`
```
**Why `ROW_NUMBER` instead of `RANK`?** Ask the model to add a 2-sentence contrast.

In [9]:
sql = f"""
WITH subcat_sales AS (
    -- CTE 1: Calculate total sales for each sub-category within each region
    SELECT
        Region,
        Sub_Category,
        SUM(Sales) AS total_sales
    FROM
        `upbeat-aspect-471118-v8.lab01_dataset.superstore_clean`
    GROUP BY
        Region,
        Sub_Category
),
ranked_subcat_sales AS (
    -- CTE 2: Rank sub-categories within each region by their total sales
    SELECT
        Region,
        Sub_Category,
        total_sales,
        ROW_NUMBER() OVER (PARTITION BY Region ORDER BY total_sales DESC) AS rn
    FROM
        subcat_sales
)
-- Final SELECT: Filter to get the single highest revenue sub-category per region
SELECT
    Region,
    Sub_Category,
    total_sales
FROM
    ranked_subcat_sales
WHERE
    rn = 1
ORDER BY
    Region ASC
"""

# Execute the query
query_job = client.query(sql)

# Convert results to a Pandas DataFrame for easy display
results_df = query_job.to_dataframe()

# Display the DataFrame in Colab
display(results_df)

,Region,Sub_Category,total_sales
0,Central,Chairs,85230.646
1,East,Phones,100614.982
2,South,Phones,58304.438
3,West,Chairs,101781.328


### E2. YoY growth with LAG
**Prompt:**
```
BigQuery SQL only.
Task: Compute year-over-year revenue growth for 'Phones' sub-category.
Table: `[YOUR_PROJECT].superstore_data.sales`
Steps:
- Filter to `Sub_Category = 'Phones'`
- Aggregate yearly revenue using EXTRACT(YEAR FROM Order_Date)
- Add `LAG(yearly_revenue) OVER (ORDER BY year)` as `prev_revenue`
- Compute `yoy_pct = 100.0 * (yearly_revenue - prev_revenue) / prev_revenue`
Output: `year`, `yearly_revenue`, `prev_revenue`, `yoy_pct`
Sort by `year` ASC
```
**Ask for**: a guard against divide-by-zero or NULL previous year.

In [ ]:
sql = f"""
WITH yearly_phone_sales AS (
    -- Step 1 & 2: Filter for 'Phones' and aggregate yearly revenue
    SELECT
        EXTRACT(YEAR FROM order_date) AS year,
        SUM(sales) AS yearly_revenue
    FROM
        `upbeat-aspect-471118-v8.lab01_dataset.superstore_clean`
    WHERE
        sub_category = 'Phones'
    GROUP BY
        year
),
lagged_sales AS (
    -- Step 3: Add the previous year's revenue using LAG window function
    SELECT
        year,
        yearly_revenue,
        LAG(yearly_revenue, 1, 0) OVER (ORDER BY year) AS prev_revenue -- Use 0 as default for the first year
    FROM
        yearly_phone_sales
)
-- Step 4 & 5: Compute YoY percentage growth and select final columns
SELECT
    year,
    yearly_revenue,
    prev_revenue,
    -- Calculate YoY percentage, handle division by zero (prev_revenue = 0 or NULL)
    CASE
        WHEN prev_revenue IS NULL OR prev_revenue = 0 THEN NULL -- Or 'INF'/'NaN' if preferred, but NULL is standard
        ELSE ROUND(100.0 * (yearly_revenue - prev_revenue) / prev_revenue, 2)
    END AS yoy_pct
FROM
    lagged_sales
ORDER BY
    year ASC
"""

# Execute the query
query_job = client.query(sql)

# Convert results to a Pandas DataFrame for easy display
results_df = query_job.to_dataframe()

# Display the DataFrame in Colab
display(results_df)

,year,yearly_revenue,prev_revenue,yoy_pct
0,2014,77390.806,0.000,NaN
1,2015,68313.702,77390.806,-11.73
2,2016,78962.030,68313.702,15.59
3,2017,105340.516,78962.030,33.41


### E3. 3‑month moving average (MA)
**Prompt:**
```
BigQuery SQL only.
Task: For the 'Corporate' segment, compute a 3-month moving average of monthly revenue.
Table: `[YOUR_PROJECT].superstore_data.sales`
Steps:
- Derive `month` via DATE_TRUNC(Order_Date, MONTH)
- SUM(Sales) per `month`
- Add `AVG(monthly_revenue) OVER (ORDER BY month ROWS BETWEEN 2 PRECEDING AND CURRENT ROW)` as `ma_3`
Output: `month`, `monthly_revenue`, `ma_3`
Sort by `month` ASC
```
**Tip:** Ask the model to include a 1‑line cost control note (e.g., restrict date range while iterating).

In [ ]:
sql = f"""
WITH monthly_revenue_by_segment AS (
    -- Step 1 & 2: Derive month and SUM(Sales) per month and segment
    SELECT
        Segment,
        DATE_TRUNC(Order_Date, MONTH) AS month,
        SUM(Sales) AS monthly_revenue
    FROM
        `upbeat-aspect-471118-v8.lab01_dataset.superstore_clean`
    WHERE
        Segment = 'Corporate' -- Filter for the 'Corporate' segment
    GROUP BY
        Segment,
        month
),
monthly_revenue_with_lag AS (
    -- Step 3: Add the 3-month moving average using a window function
    SELECT
        month,
        monthly_revenue,
        AVG(monthly_revenue) OVER (ORDER BY month ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS ma_3
    FROM
        monthly_revenue_by_segment
)
-- Final SELECT: Output month, monthly_revenue, and the 3-month moving average
SELECT
    month,
    monthly_revenue,
    ma_3
FROM
    monthly_revenue_with_lag
ORDER BY
    month ASC
"""

# Execute the query
query_job = client.query(sql)

# Convert results to a Pandas DataFrame for easy display
results_df = query_job.to_dataframe()

# Display the DataFrame in Colab
display(results_df)

,month,monthly_revenue,ma_3
0,2014-01-01,1701.5280,1701.528000
1,2014-02-01,1183.6680,1442.598000
2,2014-03-01,11106.7990,4663.998333
3,2014-04-01,14131.7290,8807.398667
4,2014-05-01,9142.0000,11460.176000
5,2014-06-01,3970.9140,9081.547667
6,2014-07-01,10032.9880,7715.300667
7,2014-08-01,7451.7740,7151.892000
8,2014-09-01,15507.7450,10997.502333
9,2014-10-01,12637.6780,11865.732333


## Part F — Debugging & Optimization Prompts
**Aim:** Use the model as a rubber duck for error handling and performance.

### F1. Explain the error, propose a fix
**Prompt:**
```
I ran this BigQuery SQL and got an error:
[PASTE ERROR MESSAGE and the exact SQL here]
Act as a BigQuery trouble‑shooter.
1) Identify the root cause.
2) Propose the smallest possible fix.
3) Suggest a quick sanity check query to verify the fix.
Return only the corrected SQL and a 2‑sentence rationale.
```

WITH subcat_sales AS (
    -- CTE 1: Calculate total sales for each sub-category within each region
    SELECT
        Region,
        Sub_Category,
        SUM(Sales) AS total_sales
    FROM
        `upbeat-aspect-471118-v8.lab01_dataset.superstore_clean`
    GROUP BY
        Region,
        Sub_Category
),
ranked_subcat_sales AS (
    -- CTE 2: Rank sub-categories within each region by their total sales
    SELECT
        Region,
        Sub_Category,
        total_sales,
        ROW_NUMBER() OVER (PARTITION BY Region ORDER BY total_sales DESC) AS rn
    FROM
        subcat_sales
)
-- Final SELECT: Filter to get the single highest revenue sub-category per region
SELECT
    Region,
    Sub_Category,
    total_sales -- Removed AVG() as it's not needed when filtering for rn = 1
FROM
    ranked_subcat_sales
WHERE
    rn = 1
ORDER BY
    Region ASC
Rationale: The error "SELECT list expression references column Region which is neither grouped nor aggregated" occurred because AVG(total_sales) was used in the final SELECT statement without grouping or aggregating Region and Sub_Category. Since we are filtering for rn = 1 (which selects a single row per region), we just need to select Region, Sub_Category, and the total_sales from that ranked row, not aggregate them further.

### F2. Reduce cost / improve speed
**Prompt:**
```
Act as a BigQuery cost optimizer.
Given this query (below), list 3 ways to reduce scanned bytes and improve performance without changing the business logic.
[PASTE YOUR SQL HERE]
Prioritize: partition filters, column pruning, pre-aggregations, and temporary results via CTEs.
```

Partition Filter on Order_Date:

Rationale: The query implicitly scans all data to calculate the average of Sales and retrieve Order_Date. If the table is partitioned by Order_Date (or a date-like column), filtering on a specific date range in the WHERE clause will significantly reduce scanned bytes.
Modification: Add a DATE_TRUNC or date range filter to the WHERE clause.
Example:
SELECT
    Segment,
    AVG(Sales) AS average_sales,
    Order_Date -- Still selecting Order_Date, but it will be for specific partitions
FROM
    `upbeat-aspect-471118-v8.lab01_dataset.superstore_clean`
WHERE
    Segment = "Binders"
    AND DATE_TRUNC(Order_Date, MONTH) = '2023-01-01' -- Example: Filter for January 2023
GROUP BY
    Segment, Order_Date -- GROUP BY Order_Date as it's in SELECT list and not aggregated
Note: The original query implies Segment = "Binders" is the only filter. If you intended to get AVG(Sales) for all of "Binders" across all dates, selecting Order_Date without aggregation or a filter would still scan extensively. If Order_Date is needed for context after aggregation, it should be aggregated or the query logic adjusted. The example above makes a reasonable assumption for illustration.
Column Pruning with SELECT List:

Rationale: The original query selects Segment, AVG(Sales), and Order_Date. However, Order_Date is not aggregated and will be returned as a single value (likely the Order_Date from one of the rows if Order_Date were also grouped, or an error if not). If Order_Date is not strictly required in the output, removing it will prevent BigQuery from having to process that column's data.
Modification: Remove Order_Date from the SELECT list if it's not essential for the final output.
Example:
SELECT
    Segment,
    AVG(Sales) AS average_sales
    -- Removed Order_Date
FROM
    `upbeat-aspect-471118-v8.lab01_dataset.superstore_clean`
WHERE
    Segment = "Binders"
GROUP BY
    Segment
Note: If Order_Date is genuinely needed in the output, but for specific records (not an average of dates), the query's purpose needs re-evaluation. For example, you might want AVG(Sales) for each month, in which case Order_Date would be truncated and then grouped.
Pre-aggregation using a CTE (for potential reuse or simplification):

Rationale: While not strictly reducing bytes for this exact query if Order_Date is kept and not aggregated, a CTE can simplify and potentially optimize if the aggregation itself is complex or if you wanted to add further filters after aggregation. For this specific case, if we omit Order_Date from the final SELECT, the CTE makes the aggregation explicit. If Order_Date must be selected, pre-aggregation to monthly revenue would be the way to go if Order_Date were part of the GROUP BY.
Modification: Use a CTE to perform the aggregation first, then select from it. This is more about structuring for clarity and potential optimization if complex filtering or joins were to follow.
Example (assuming Order_Date is removed from SELECT for optimization):
WITH binders_monthly_sales AS (
    SELECT
        DATE_TRUNC(Order_Date, MONTH) AS month, -- Truncate Order_Date to group by month
        AVG(Sales) AS monthly_average_sales
    FROM
        `upbeat-aspect-471118-v8.lab01_dataset.superstore_clean`
    WHERE
        Segment = "Binders"
    GROUP BY
        month -- Group by the truncated month
)
SELECT
    month, -- Now selecting month instead of individual Order_Date
    monthly_average_sales
FROM
    binders_monthly_sales
ORDER BY
    month ASC -- Added ordering for clarity
Note: This assumes a reasonable business intent where averaging sales per month for the "Binders" segment makes sense, and thus Order_Date is truncated and used for grouping and output. The original query's inclusion of Order_Date without aggregation or specific filtering suggests a potential misunderstanding of how AVG works across non-grouped, non-aggregated columns.


## Part G — Validation & Counter‑examples (DIVE: Validate)
**Aim:** Avoid “first‑answer fallacy” by testing alternatives.

### G1. Ask for counter‑queries
**Prompt:**
```
I concluded that 'Tables' is a high‑sales but negative‑profit sub-category due to high discounts.
Create two alternative BigQuery SQL queries that could falsify or nuance this finding:
- One that slices by region and time
- One that controls for order priority or ship mode
Return BigQuery SQL only, then a one-paragraph note on how to compare outcomes.
```

-- Query 1: Slice by Region and Time
WITH subcat_sales_profit AS (
    SELECT
        Region,
        EXTRACT(YEAR FROM Order_Date) AS order_year,
        EXTRACT(MONTH FROM Order_Date) AS order_month,
        Sub_Category,
        SUM(Sales) AS total_sales,
        SUM(Profit) AS total_profit
    FROM
        `upbeat-aspect-471118-v8.lab01_dataset.superstore_clean`
    WHERE
        Sub_Category = 'Tables' -- Focusing on the sub-category in question
    GROUP BY
        Region,
        order_year,
        order_month,
        Sub_Category
)
SELECT
    Region,
    order_year,
    order_month,
    total_sales,
    total_profit
FROM
    subcat_sales_profit
ORDER BY
    Region,
    order_year,
    order_month;

-- Query 2: Control for Order Priority or Ship Mode
WITH subcat_sales_profit AS (
    SELECT
        Sub_Category,
        Order_Priority, -- Or Ship_Mode, depending on which is more relevant/available
        SUM(Sales) AS total_sales,
        SUM(Profit) AS total_profit
    FROM
        `upbeat-aspect-471118-v8.lab01_dataset.superstore_clean`
    WHERE
        Sub_Category = 'Tables' -- Focusing on the sub-category in question
    GROUP BY
        Sub_Category,
        Order_Priority -- Or Ship_Mode
)
SELECT
    Sub_Category,
    Order_Priority, -- Or Ship_Mode
    total_sales,
    total_profit
FROM
    subcat_sales_profit
ORDER BY
    Order_Priority, -- Or Ship_Mode
    total_profit ASC;
To compare outcomes, examine the total_profit column in each query's results. If Query 1 shows that the negative profit for 'Tables' is concentrated in specific regions or months, or if it's consistently negative across all periods, it suggests regional or temporal factors influencing profitability. If Query 2 reveals that 'Tables' consistently underperforms across all Order_Priority levels or Ship_Modes, it strengthens the discount hypothesis; however, if specific priorities (e.g., 'High') show positive profit while others are negative, it suggests that certain order types might be less susceptible to discount-driven losses.

## Part H — Synthesis (DIVE: Extend)
**Aim:** Turn analysis into business‑ready insights.

### H1. Executive‑style summary
**Prompt:**
```
Act as a business strategist.
Based on the following metrics/figures (briefly summarize your results here), write a 4-sentence executive summary:
- 1 sentence: what changed and by how much
- 1 sentence: why it likely changed (drivers)
- 1 sentence: recommended action (who/what/when)
- 1 sentence: metric to monitor next
```

The overall year-over-year revenue growth was positive, driven primarily by strong performance in the Phones category, which saw significant increases. This growth is likely attributed to increased demand and potentially successful promotional strategies in the East and South regions where Phones are the top sellers, while Chairs remain dominant in Central and West. We recommend an immediate focus on optimizing the profitability of Chairs in Central and West regions through targeted inventory management and pricing strategies to capitalize on their high sales volume. Moving forward, we will closely monitor the YoY growth of the Phones category across all regions, particularly in East and South, to ensure sustained momentum.

### H2. Convert final SQL into an automated job (optional)
**Prompt (use only after your SQL is final):**
```
Convert my final BigQuery SQL into a Python script that can run as a scheduled job from Colab or Cloud Functions.
Requirements:
- Use python‑bigquery client
- Parameterize date range
- Write results to a destination table `[YOUR_PROJECT].analytics.outputs_kpi`
- Add basic error handling & logging
Return one complete runnable script.
```

---
## Submission checklist
- [ ] Kept prompts precise and reproducible  
- [ ] Captured at least **one** CTE query and **one** window function query  
- [ ] Documented **two** validation attempts (counter‑queries or alternate slice)  
- [ ] Wrote a 4‑sentence executive summary based on results  
- [ ] (Optional) Converted final query into a scheduled job
---